## Model Architecture

### Model with input layer with 30 neurons (since =30 input features) , a hidden layer(that uses Relu fxn) with 20 neuron and a output layer(uses sigmoid fxn) with single **output**

##### So total weights = layer1 (30 x 20) + layer2( 20 x 1) = 620
##### SO total bias = 20 + 1 = 21

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import torch
import torch.nn as nn
import torch.optim

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder

In [3]:
df = pd.read_csv('https://raw.githubusercontent.com/gscdit/Breast-Cancer-Detection/master/data.csv')
df.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


In [4]:
df.drop(columns=['id','Unnamed: 32'],inplace=True)

In [5]:
x = df.iloc[:,1:]
y = df.iloc[:,:1]

In [6]:
# Train test split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=42)

In [7]:
# Scaling
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

In [8]:
# Label encoder
encoder = LabelEncoder()
y_train = encoder.fit_transform(y_train)
y_test = encoder.transform(y_test)

/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_label.py:110: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_label.py:129: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


In [9]:
# Converting the Numpy arrays to pytorch tensors
x_train_tensor = torch.tensor(x_train).float()
x_test_tensor = torch.from_numpy(x_test).float()
y_train_tensor = torch.from_numpy(y_train).float()
y_test_tensor = torch.from_numpy(y_test).float()

In [10]:
x_train_tensor.shape

torch.Size([455, 30])

# Define the model

In [11]:
class NeuralNetwork(nn.Module): # Must inherit the nn.Module class inorder to use "nn module"

  # Constructor
  def __init__(self,num_features):

    super().__init__()  #----> invoking the parent class(nn.module) constructor using super keyword

    # Creating a Sequential container for the neural n/w
    self.network = nn.Sequential(
        nn.Linear(num_features,20),  # (hidden layer) Accepts 2 paramaters , (numer_of_input_features , number_of_outputs)
        nn.ReLU(),                    # Activation fxn
        nn.Linear(20,1),              # Output layer
        nn.Sigmoid()
    )

  def forward(self,features):
    out = self.network(features)
    return out



### Important Parameters

In [12]:
learning_rate = 0.1
epochs = 25

In [13]:
# Create a loss function using nn.module
loss_function = nn.BCELoss() # Binary loss entropy

# Training pipeline


In [14]:
# create model
model = NeuralNetwork(x_train_tensor.shape[1])

# Create a Optimer(stochastic gradient descent)
optimizer = torch.optim.SGD(model.parameters(),lr=learning_rate) # Takes 2 inputs paramater i.e (all_models_paramaters, learning_rate)

# define loop
for epoch in range(epochs):

  # forward pass
  y_pred = model.forward(x_train_tensor)

  # Loss calculate
  loss = loss_function(y_pred,y_train_tensor.reshape(-1,1)) #Reshaping the data

  # clear gradients
  optimizer.zero_grad()

  #backward pass
  loss.backward()

  #Paramaters update
  optimizer.step()




 # print loss in each epoch
  print(f'Epoch: {epoch + 1}, Loss: {loss.item()}')


Epoch: 1, Loss: 0.7675979137420654
Epoch: 2, Loss: 0.7387908697128296
Epoch: 3, Loss: 0.7131224274635315
Epoch: 4, Loss: 0.6899362206459045
Epoch: 5, Loss: 0.6684759259223938
Epoch: 6, Loss: 0.6483823657035828
Epoch: 7, Loss: 0.6292967200279236
Epoch: 8, Loss: 0.6109362840652466
Epoch: 9, Loss: 0.5930449366569519
Epoch: 10, Loss: 0.575520932674408
Epoch: 11, Loss: 0.558265209197998
Epoch: 12, Loss: 0.5411900877952576
Epoch: 13, Loss: 0.5242103338241577
Epoch: 14, Loss: 0.5072535276412964
Epoch: 15, Loss: 0.49040085077285767
Epoch: 16, Loss: 0.4736494719982147
Epoch: 17, Loss: 0.4571450650691986
Epoch: 18, Loss: 0.4408573806285858
Epoch: 19, Loss: 0.4248286187648773
Epoch: 20, Loss: 0.40914949774742126
Epoch: 21, Loss: 0.39389753341674805
Epoch: 22, Loss: 0.379148006439209
Epoch: 23, Loss: 0.3648957312107086
Epoch: 24, Loss: 0.35117724537849426
Epoch: 25, Loss: 0.33805176615715027


In [15]:
# model evaluation
with torch.no_grad():
  y_pred = model.forward(x_test_tensor)
  y_pred = (y_pred > 0.5).float()
  accuracy = (y_pred == y_test_tensor).float().mean()
  print(f'Accuracy: {accuracy.item()}')


Accuracy: 0.5366266369819641


In [16]:
# To access the weigths
model.network[2].weight

Parameter containing:
tensor([[ 0.1240,  0.2462,  0.1637, -0.1832, -0.0301,  0.1201, -0.0116, -0.2972,
          0.3727, -0.0903,  0.4134, -0.3128,  0.1174, -0.1484,  0.1347,  0.0572,
          0.0038, -0.3655,  0.0596,  0.2007]], requires_grad=True)

# To visualize the network

In [17]:
!pip install torchinfo

In [18]:
from torchinfo import summary

summary(model, input_size=x_train_tensor.shape) #Input size is no_of_rows and columns

Layer (type:depth-idx)                   Output Shape              Param #
NeuralNetwork                            [455, 1]                  --
├─Sequential: 1-1                        [455, 1]                  --
│    └─Linear: 2-1                       [455, 20]                 620
│    └─ReLU: 2-2                         [455, 20]                 --
│    └─Linear: 2-3                       [455, 1]                  21
│    └─Sigmoid: 2-4                      [455, 1]                  --
Total params: 641
Trainable params: 641
Non-trainable params: 0
Total mult-adds (Units.MEGABYTES): 0.29
Input size (MB): 0.05
Forward/backward pass size (MB): 0.08
Params size (MB): 0.00
Estimated Total Size (MB): 0.13